In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    BBB_Train_Decile3 = pd.read_pickle("BBB_Train_Decile3.pkl")
    BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("BBB_Test_Decile3.pkl")
    y_test = pd.read_pickle("BBB_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(BBB_Train_Decile3, BBB_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150, 200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM BBB Decile 3')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))
    

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

Epoch 38/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4929 - acc: 0.7762 - val_loss: 0.4757 - val_acc: 0.7850
Epoch 39/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4877 - acc: 0.7817 - val_loss: 0.4768 - val_acc: 0.7915
Epoch 40/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4902 - acc: 0.7875 - val_loss: 0.4744 - val_acc: 0.7876
Epoch 41/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4891 - acc: 0.7865 - val_loss: 0.4759 - val_acc: 0.7902
Epoch 42/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4858 - acc: 0.7846 - val_loss: 0.4765 - val_acc: 0.7940
Epoch 43/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4870 - acc: 0.7885 - val_loss: 0.4721 - val_acc: 0.7876
Epoch 44/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4870 - acc: 0.7868 - val_loss: 0.4727 - val_acc: 0.7927
Epoch 45/100
3087/3087 [========================

Epoch 98/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4709 - acc: 0.7924 - val_loss: 0.4663 - val_acc: 0.7915
Epoch 99/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4694 - acc: 0.7969 - val_loss: 0.4662 - val_acc: 0.7850
Epoch 100/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4715 - acc: 0.7933 - val_loss: 0.4663 - val_acc: 0.7927
Test accuracy: 0.7927461139896373
Train on 3087 samples, validate on 772 samples
Epoch 1/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.6296 - acc: 0.6388 - val_loss: 0.5801 - val_acc: 0.6878
Epoch 2/150
3087/3087 [==============================] - 3s 1ms/step - loss: 0.5596 - acc: 0.7133 - val_loss: 0.5167 - val_acc: 0.7422
Epoch 3/150
3087/3087 [==============================] - 3s 1ms/step - loss: 0.5324 - acc: 0.7538 - val_loss: 0.5090 - val_acc: 0.7694
Epoch 4/150
3087/3087 [==============================] - 3s 1ms/step - loss: 0.5237 - acc: 0.7638 - val_l

3087/3087 [==============================] - 3s 1ms/step - loss: 0.4569 - acc: 0.7937 - val_loss: 0.4595 - val_acc: 0.7850
Epoch 57/150
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4545 - acc: 0.7995 - val_loss: 0.4595 - val_acc: 0.7850
Epoch 58/150
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4606 - acc: 0.7956 - val_loss: 0.4595 - val_acc: 0.7850
Epoch 59/150
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4526 - acc: 0.7946 - val_loss: 0.4596 - val_acc: 0.7850
Epoch 60/150
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4519 - acc: 0.7949 - val_loss: 0.4595 - val_acc: 0.7863
Epoch 61/150
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4485 - acc: 0.7966 - val_loss: 0.4595 - val_acc: 0.7876
Epoch 62/150
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4522 - acc: 0.7959 - val_loss: 0.4596 - val_acc: 0.7876
Epoch 63/150
3087/3087 [==============================] - 3s 

Epoch 25/150
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4581 - acc: 0.7949 - val_loss: 0.4650 - val_acc: 0.7876
Epoch 26/150
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4589 - acc: 0.7937 - val_loss: 0.4651 - val_acc: 0.7798
Epoch 27/150
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4593 - acc: 0.7988 - val_loss: 0.4655 - val_acc: 0.7798
Epoch 28/150
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4597 - acc: 0.7940 - val_loss: 0.4657 - val_acc: 0.7850
Epoch 29/150
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4583 - acc: 0.7975 - val_loss: 0.4667 - val_acc: 0.7876

Epoch 00029: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 30/150
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4577 - acc: 0.7959 - val_loss: 0.4663 - val_acc: 0.7876
Epoch 31/150
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4537 - acc: 0.8014 - val_loss: 0.4660 -

...

Epoch 147/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4572 - acc: 0.7969 - val_loss: 0.4646 - val_acc: 0.7837
Epoch 148/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4551 - acc: 0.7943 - val_loss: 0.4644 - val_acc: 0.7837
Epoch 149/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4542 - acc: 0.7988 - val_loss: 0.4644 - val_acc: 0.7837
Epoch 150/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4544 - acc: 0.8017 - val_loss: 0.4646 - val_acc: 0.7837
Epoch 151/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4572 - acc: 0.7943 - val_loss: 0.4643 - val_acc: 0.7837
Epoch 152/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4502 - acc: 0.7982 - val_loss: 0.4643 - val_acc: 0.7837
Epoch 153/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4543 - acc: 0.7992 - val_loss: 0.4643 - val_acc: 0.7837
Epoch 154/200
3087/3087 [================

Epoch 66/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4933 - acc: 0.7898 - val_loss: 0.4840 - val_acc: 0.8031
Epoch 67/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4893 - acc: 0.7901 - val_loss: 0.4840 - val_acc: 0.8018
Epoch 68/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4875 - acc: 0.7956 - val_loss: 0.4841 - val_acc: 0.8018
Epoch 69/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4888 - acc: 0.7878 - val_loss: 0.4839 - val_acc: 0.8031
Epoch 70/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4907 - acc: 0.7904 - val_loss: 0.4839 - val_acc: 0.8018
Epoch 71/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4903 - acc: 0.7891 - val_loss: 0.4843 - val_acc: 0.8018
Epoch 72/100
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4937 - acc: 0.7924 - val_loss: 0.4841 - val_acc: 0.8018
Epoch 73/100
3087/3087 [========================

3087/3087 [==============================] - 3s 1ms/step - loss: 0.4570 - acc: 0.8011 - val_loss: 0.4617 - val_acc: 0.7876
Epoch 26/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4587 - acc: 0.7988 - val_loss: 0.4617 - val_acc: 0.7863
Epoch 27/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4617 - acc: 0.7998 - val_loss: 0.4617 - val_acc: 0.7863
Epoch 28/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4594 - acc: 0.7985 - val_loss: 0.4617 - val_acc: 0.7863
Epoch 29/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4579 - acc: 0.7992 - val_loss: 0.4617 - val_acc: 0.7863

Epoch 00029: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 30/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4565 - acc: 0.7969 - val_loss: 0.4617 - val_acc: 0.7863
Epoch 31/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4601 - acc: 0.7959 - val_loss: 0.4617 - val_acc: 0.7

Epoch 85/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4627 - acc: 0.8001 - val_loss: 0.4619 - val_acc: 0.7863
Epoch 86/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4576 - acc: 0.8017 - val_loss: 0.4619 - val_acc: 0.7876
Epoch 87/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4620 - acc: 0.7985 - val_loss: 0.4619 - val_acc: 0.7876
Epoch 88/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4549 - acc: 0.7972 - val_loss: 0.4619 - val_acc: 0.7876
Epoch 89/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4587 - acc: 0.7992 - val_loss: 0.4619 - val_acc: 0.7850
Epoch 90/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4587 - acc: 0.8005 - val_loss: 0.4619 - val_acc: 0.7850
Epoch 91/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4586 - acc: 0.8001 - val_loss: 0.4618 - val_acc: 0.7850
Epoch 92/200
3087/3087 [========================

Epoch 145/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4579 - acc: 0.7959 - val_loss: 0.4616 - val_acc: 0.7863
Epoch 146/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4558 - acc: 0.7985 - val_loss: 0.4616 - val_acc: 0.7876
Epoch 147/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4602 - acc: 0.7995 - val_loss: 0.4616 - val_acc: 0.7876
Epoch 148/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4551 - acc: 0.8001 - val_loss: 0.4616 - val_acc: 0.7863
Epoch 149/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4591 - acc: 0.8011 - val_loss: 0.4616 - val_acc: 0.7863
Epoch 150/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4519 - acc: 0.7979 - val_loss: 0.4616 - val_acc: 0.7863
Epoch 151/200
3087/3087 [==============================] - 3s 1ms/step - loss: 0.4545 - acc: 0.7988 - val_loss: 0.4616 - val_acc: 0.7876
Epoch 152/200
3087/3087 [================

In [3]:
0.8297

0.8297